<a href="https://colab.research.google.com/github/Amanda9805/Detecting-Machine-Generated-Texts/blob/development/COS_760_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade datasets fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [ ]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

In [76]:
import pandas as pd
import re
import nltk
from huggingface_hub import login
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from datasets import load_dataset


#shona_data = load_dataset("DigitalUmuganda/AfriVoice", "sn", streaming=True, split="train[:10%]")

# english_data = load_dataset("oscar-corpus/OSCAR-2201", language="en", streaming=True)

# num_samples = 500
# samples = []
# for i, example in enumerate(english_data["train"]):
#     if i >= num_samples:
#         break
#     samples.append(example['text'])

# eng_df = pd.DataFrame(samples, columns=['text'])
# eng_df.head()

zulu_data = load_dataset("dsfsi/vukuzenzele-monolingual", "zul")

def reformatData(dataDict):
  langList = []
  for index, row in dataDict.iterrows():
    # Access data using column names from the row Series
    if 'text' in row and row['text']:
        langList.append({
        'text': row.get('text', ''),
        'author': row.get('author', ''),
        'title': row.get('title', ''),
        'language': 'zul'
              })

  return langList

human_data = reformatData(zulu_data['train'].to_pandas())

zul_df = pd.DataFrame(human_data)
zul_df['cleaned_text'] = zul_df['text'].apply(clean_text)
zul_df['tokens'] = zul_df['cleaned_text'].apply(word_tokenize)
zul_df['label'] = 0
zul_df.head()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,text,author,title,language,cleaned_text,tokens,label
0,Ngokusho kukaNgqongqoshe Wezempilo uDkt. Joe P...,Allison Cooper\n,Zivikele kwi-COVID-19 ngaphambi kobusika \n,zul,ngokusho kukangqongqoshe wezempilo udkt joe ph...,"[ngokusho, kukangqongqoshe, wezempilo, udkt, j...",0
1,Abafundi BakaMa tikuletsheni bakulonyaka kumel...,Dale Hes \n,Kwethulwe ngokusemthethweni uhlelo lweWoza Ma...,zul,abafundi bakama tikuletsheni bakulonyaka kumel...,"[abafundi, bakama, tikuletsheni, bakulonyaka, ...",0
2,Lolu hlelo luhlose ukuthuthukisa ikhono labant...,Vukuzenzele Unnamed\n,Uhlelo lokudla esikoleni luqinisekisa ukuthi a...,zul,lolu hlelo luhlose ukuthuthukisa ikhono labant...,"[lolu, hlelo, luhlose, ukuthuthukisa, ikhono, ...",0
3,hulumeni eNingizimu Afrika yonkana abakwazi uk...,More Matshediso\n,Izisu ezigcwele ziholela emiphumeleni engcono\n,zul,hulumeni eningizimu afrika yonkana abakwazi uk...,"[hulumeni, eningizimu, afrika, yonkana, abakwa...",0
4,Kamuva nje bengivakashele eThekwini KwaZulu-Na...,Vukuzenzele Unnamed\n,Ukwakha kabusha impilo nendlela yokuphila emuv...,zul,kamuva nje bengivakashele ethekwini kwazulunat...,"[kamuva, nje, bengivakashele, ethekwini, kwazu...",0


In [77]:
# eng_df['cleaned_text'] = eng_df['text'].apply(clean_text)
# eng_df['tokens'] = eng_df['cleaned_text'].apply(word_tokenize)
# print(eng_df.head())

zul_df['cleaned_text'] = zul_df['text'].apply(clean_text)
zul_df['tokens'] = zul_df['cleaned_text'].apply(word_tokenize)
zul_df.head()

Human-written texts: 126


,text,author,title,language,cleaned_text,tokens,label
0,Ngokusho kukaNgqongqoshe Wezempilo uDkt. Joe P...,Allison Cooper\n,Zivikele kwi-COVID-19 ngaphambi kobusika \n,zul,ngokusho kukangqongqoshe wezempilo udkt joe ph...,"[ngokusho, kukangqongqoshe, wezempilo, udkt, j...",0
1,Abafundi BakaMa tikuletsheni bakulonyaka kumel...,Dale Hes \n,Kwethulwe ngokusemthethweni uhlelo lweWoza Ma...,zul,abafundi bakama tikuletsheni bakulonyaka kumel...,"[abafundi, bakama, tikuletsheni, bakulonyaka, ...",0
2,Lolu hlelo luhlose ukuthuthukisa ikhono labant...,Vukuzenzele Unnamed\n,Uhlelo lokudla esikoleni luqinisekisa ukuthi a...,zul,lolu hlelo luhlose ukuthuthukisa ikhono labant...,"[lolu, hlelo, luhlose, ukuthuthukisa, ikhono, ...",0
3,hulumeni eNingizimu Afrika yonkana abakwazi uk...,More Matshediso\n,Izisu ezigcwele ziholela emiphumeleni engcono\n,zul,hulumeni eningizimu afrika yonkana abakwazi uk...,"[hulumeni, eningizimu, afrika, yonkana, abakwa...",0
4,Kamuva nje bengivakashele eThekwini KwaZulu-Na...,Vukuzenzele Unnamed\n,Ukwakha kabusha impilo nendlela yokuphila emuv...,zul,kamuva nje bengivakashele ethekwini kwazulunat...,"[kamuva, nje, bengivakashele, ethekwini, kwazu...",0


In [78]:
from transformers import pipeline
import json

en_generator = pipeline('text-generation', model='gpt2')


english_prompts = [
    "Explain the significance of lobola in Southern Africa",
    "Write a short dialogue between two friends in Johannesburg",
    "Describe linguistic features that make isiZulu agglutinative"
]

zulu_prompts = [
    "Chaza ngokubaluleka kwesiko lwelobola eNingizimu Afrika",
    "Bhala inkulumo emfushane phakathi kwabangani ababili eGoli",
    "Landela indaba yamaZulu ngokomlando",
    "Chaza ngamasiko amasha eZulu eskhathini samanje",
    "Bhala inganekwane ethi 'UNogwaja noFudu'"
]

# def generate_with_prompts(prompts, generator, language, samples_per_prompt=3):
#     data = []
#     for prompt in prompts:
#         for _ in range(samples_per_prompt):
#             output = generator(prompt, max_length=100, do_sample=True, temperature=0.7)
#             data.append({
#                 'prompt': prompt,
#                 'text': output[0]['generated_text'],
#                 'label': 'machine',
#                 'language': language,
#                 'prompt_type': 'cultural' if "tsika" in prompt else 'linguistic'  # Tag for analysis
#             })
#     return pd.DataFrame(data)


# eng_mg_df = generate_with_prompts(english_prompts, en_generator, 'English')
# eng_mg_df['cleaned_text'] = eng_mg_df['text'].apply(clean_text)
# eng_mg_df['tokens'] = eng_mg_df['cleaned_text'].apply(word_tokenize)
# print(eng_mg_df['text'].iloc[0][:300])

def load_jsonl_data(file_path):
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                line = line.strip()
                if line:  # Skip empty lines
                    try:
                        data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"Error parsing JSON line: {e}")
                        continue
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []

    return data

zul_mg_df = pd.DataFrame(load_jsonl_data('zulu_mg_text.json'))

machine_data =reformatData(zul_mg_df)
zul_mg_df = pd.DataFrame(machine_data)
zul_mg_df['text'] = zul_mg_df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
zul_mg_df['language'] = 'zul'
zul_mg_df['cleaned_text'] = zul_mg_df['text'].apply(clean_text)
zul_mg_df['tokens'] = zul_mg_df['cleaned_text'].apply(word_tokenize)
zul_mg_df['label'] = 1

zul_mg_df.head()

Device set to use cpu


,text,author,title,language,cleaned_text,tokens,label
0,Uhulumeni uzivezile izinhlelo ezintsha zokwakh...,Sipho Mthembu,Uhulumeni Uvula Uhlelo Olusha Lwezindlu,zul,uhulumeni uzivezile izinhlelo ezintsha zokwakh...,"[uhulumeni, uzivezile, izinhlelo, ezintsha, zo...",1
1,Izinkulungwane zabantu ziye zahlangana namuhla...,Nomsa Dlamini,Abantu Bayaqhubeka Ukusola Ngamanani Kaphethiloli,zul,izinkulungwane zabantu ziye zahlangana namuhla...,"[izinkulungwane, zabantu, ziye, zahlangana, na...",1
2,Ithimba leKaizer Chiefs lifinyelele kokugcina ...,Mandla Zulu,Ithimba LeKaizer Chiefs Lingene Kokugcina KweN...,zul,ithimba lekaizer chiefs lifinyelele kokugcina ...,"[ithimba, lekaizer, chiefs, lifinyelele, kokug...",1
3,UMnyango Wezempilo ufuna ukukhumbuza abantu ng...,Dr. Zanele Ngcobo,Ukubaluleka Kokujikelezwa Kwesifo SeCovid-19,zul,umnyango wezempilo ufuna ukukhumbuza abantu ng...,"[umnyango, wezempilo, ufuna, ukukhumbuza, aban...",1
4,Ucwaningo olusha luveza ukuthi ukuntuleka kwam...,Prof. Nomthandazo Cele,Ukutholakala Kwamanzi Ahlanzekile Kuyisihloko ...,zul,ucwaningo olusha luveza ukuthi ukuntuleka kwam...,"[ucwaningo, olusha, luveza, ukuthi, ukuntuleka...",1


COMBINE THE DATASETS
-------------------------

In [92]:
# Combine both datasets, this will be bad for us if we have more than 70% differnce in data length
# Combine both datasets
all_texts = pd.concat([zul_df, zul_mg_df], ignore_index=True)
all_texts_shuffled = all_texts.sample(frac=1).reset_index(drop=True)
print(f"Total texts after combining: {len(all_texts)}")
print(f"Human-written texts: {len(zul_df)}")
print(f"Machine-generated texts: {len(zul_mg_df)}")

# Create a DataFrame for easier analysis
combined_df = pd.DataFrame(all_texts_shuffled)
combined_df.head()

Total texts after combining: 166
Human-written texts: 126
Machine-generated texts: 40


,text,author,title,language,cleaned_text,tokens,label
0,Namhlanje sigubha usuku lwesizwe sethu! Ake si...,Thabo Mhlongo,Usuku Lwesizwe Luyagujwa!,zul,namhlanje sigubha usuku lwesizwe sethu ake sib...,"[namhlanje, sigubha, usuku, lwesizwe, sethu, a...",1
1,• asho ukuthi ayisekho nhlobo indlela efanelek...,Dale Hes\n,kuqedwa komshado ngokomthetho\n,zul,asho ukuthi ayisekho nhlobo indlela efanelekil...,"[asho, ukuthi, ayisekho, nhlobo, indlela, efan...",0
2,Abaholi bendabuko baneqhaza elibalulekile empi...,Dr. Nkosazana Mthembu,Indima Yabaholi Bendabuko Emiphakathini Yasema...,zul,abaholi bendabuko baneqhaza elibalulekile empi...,"[abaholi, bendabuko, baneqhaza, elibalulekile,...",1
3,UMashego odabuka eSebokeng lapha endaweni yase...,More Matshediso \n,I-NSFAS ifeze amaphu pho ami\n,zul,umashego odabuka esebokeng lapha endaweni yase...,"[umashego, odabuka, esebokeng, lapha, endaweni...",0
4,Jabulani Mzansi! Namuhla sigubha usuku lwesizw...,@ProudlySouthAfrican,Kuyajatshulelwe - Usuku Lwesizwe!,zul,jabulani mzansi namuhla sigubha usuku lwesizwe...,"[jabulani, mzansi, namuhla, sigubha, usuku, lw...",1


In [94]:
# Create a balanced dataset with equal samples from each class
# we can choose to use this of the combined one
def create_balanced_dataset(df, target_size_per_class=None):
    class_counts = df['label'].value_counts()
    min_class_size = class_counts.min()
    if target_size_per_class:
        sample_size = min(target_size_per_class, min_class_size)
    else:
        sample_size = min_class_size
    balanced_df = df.groupby('label').sample(n=sample_size, random_state=42)
    return balanced_df.reset_index(drop=True)

# Create balanced dataset, because we need the same number of samples for each class
balanced_df = create_balanced_dataset(combined_df)
print(f"\nBalanced dataset created with {len(balanced_df)} samples")
print(f"Label distribution in balanced dataset: \n{balanced_df['label'].value_counts()}")

# Save balanced dataset
balanced_output_path = 'balanced_zulu_texts.csv'
balanced_df.to_csv(balanced_output_path, index=False, encoding='utf-8')
print(f"Balanced dataset saved to: {balanced_output_path}")
balanced_df.head()


Balanced dataset created with 80 samples
Label distribution in balanced dataset: 
label
0    40
1    40
Name: count, dtype: int64
Balanced dataset saved to: balanced_zulu_texts.csv


,text,author,title,language,cleaned_text,tokens,label
0,Sekumba eqolo ukuthenga ukudla kanye nezinye i...,\n,Ukunyuka kwezindleko zokuphila kubiza ukuthi s...,zul,sekumba eqolo ukuthenga ukudla kanye nezinye i...,"[sekumba, eqolo, ukuthenga, ukudla, kanye, nez...",0
1,"Umbuso okhonayo, ose benza kahle, onomthetho n...",Vukuzenzele Unnamed\n,Imisebenzi kahulumeni eqeqeshekile\n,zul,umbuso okhonayo ose benza kahle onomthetho noq...,"[umbuso, okhonayo, ose, benza, kahle, onomthet...",0
2,Izingxoxo zezokuxhumana ezenziwa ngezobuchweph...,Kgatiso Tlhakole\n,Ukuqeqeshwa kwamahhala kanye nezinsiza zokwenz...,zul,izingxoxo zezokuxhumana ezenziwa ngezobuchweph...,"[izingxoxo, zezokuxhumana, ezenziwa, ngezobuch...",0
3,Ngokwe-DWS kube nokuntuleka kwezimvula futhi k...,More Matshediso\n,Siza ukulwisana nosuku lokuntuleka kwamanzi\n,zul,ngokwedws kube nokuntuleka kwezimvula futhi kw...,"[ngokwedws, kube, nokuntuleka, kwezimvula, fut...",0
4,Isimo esiqhubekayo kwa-Israeli kanye nase-Pal...,vukuzenzele unnamed\n,I-Israeli ne-Palestine banxuswa ukuthi bafane ...,zul,isimo esiqhubekayo kwaisraeli kanye nasepalest...,"[isimo, esiqhubekayo, kwaisraeli, kanye, nasep...",0
